<a href="https://colab.research.google.com/github/cs16b108/IB_MDDP/blob/master/IB_Diarization_Latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install webrtcvad
!pip install hmmlearn==0.2.3
!pip install python_speech_features

     |████████████████████████████████| 71kB 2.2MB/s 
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp36-cp36m-linux_x86_64.whl size=71382 sha256=954b6afb0c3f0ec873be034f031c596c9817201bddfc6d7d765c6d5927d84fcf
  Stored in directory: /root/.cache/pip/wheels/44/2a/18/bd1aec41cac7c3051fe95d92a6ed446122ea31dc713c432fa1
Successfully built webrtcvad
     |████████████████████████████████| 368kB 3.4MB/s 
  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp36-none-any.whl size=5887 sha256=71c70bd7a48aea3765ee1131ccdd3b9af26e83c63f5a052c9a6ebfbf6db34cc5
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [3]:
import os
from os.path import isfile, isdir, join
from pathlib import Path
import math
import numpy as np
# import numpy as np
import random
# import math
from scipy.stats import multivariate_normal
from matplotlib import pyplot as plt
from scipy.spatial import distance
import scipy.io.wavfile as wav
from python_speech_features import mfcc

In [4]:
########################################################
############## Read WAV and perform VAD ################
## Already done once and files stored on google drive ##
########################################################

# import collections
# import contextlib
# import sys
# import wave
# import webrtcvad


# def read_wave(path):
#     """Reads a .wav file.
#     Takes the path, and returns (PCM audio data, sample rate).
#     """
#     with contextlib.closing(wave.open(path, 'rb')) as wf:
#         num_channels = wf.getnchannels()
#         assert num_channels == 1
#         sample_width = wf.getsampwidth()
#         assert sample_width == 2
#         sample_rate = wf.getframerate()
#         assert sample_rate in (8000, 16000, 32000, 48000)
#         pcm_data = wf.readframes(wf.getnframes())
#         return pcm_data, sample_rate


# def write_wave(path, audio, sample_rate):
#     """Writes a .wav file.
#     Takes path, PCM audio data, and sample rate.
#     """
#     with contextlib.closing(wave.open(path, 'wb')) as wf:
#         wf.setnchannels(1)
#         wf.setsampwidth(2)
#         wf.setframerate(sample_rate)
#         wf.writeframes(audio)


# class Frame(object):
#     """Represents a "frame" of audio data."""
#     def __init__(self, bytes, timestamp, duration):
#         self.bytes = bytes
#         self.timestamp = timestamp
#         self.duration = duration


# def frame_generator(frame_duration_ms, audio, sample_rate):
#     """Generates audio frames from PCM audio data.
#     Takes the desired frame duration in milliseconds, the PCM data, and
#     the sample rate.
#     Yields Frames of the requested duration.
#     """
#     n = int(sample_rate * (frame_duration_ms / 1000.0) * 2)
#     offset = 0
#     timestamp = 0.0
#     duration = (float(n) / sample_rate) / 2.0
#     while offset + n < len(audio):
#         yield Frame(audio[offset:offset + n], timestamp, duration)
#         timestamp += duration
#         offset += n


# def vad_collector(sample_rate, frame_duration_ms,
#                   padding_duration_ms, vad, frames,vuv_frames):
#     """Filters out non-voiced audio frames.
#     Given a webrtcvad.Vad and a source of audio frames, yields only
#     the voiced audio.
#     Uses a padded, sliding window algorithm over the audio frames.
#     When more than 90% of the frames in the window are voiced (as
#     reported by the VAD), the collector triggers and begins yielding
#     audio frames. Then the collector waits until 90% of the frames in
#     the window are unvoiced to detrigger.
#     The window is padded at the front and back to provide a small
#     amount of silence or the beginnings/endings of speech around the
#     voiced frames.
#     Arguments:
#     sample_rate - The audio sample rate, in Hz.
#     frame_duration_ms - The frame duration in milliseconds.
#     padding_duration_ms - The amount to pad the window, in milliseconds.
#     vad - An instance of webrtcvad.Vad.
#     frames - a source of audio frames (sequence or generator).
#     Returns: A generator that yields PCM audio data.
#     """
#     num_padding_frames = int(padding_duration_ms / frame_duration_ms)
#     # We use a deque for our sliding window/ring buffer.
#     ring_buffer = collections.deque(maxlen=num_padding_frames)
#     # We have two states: TRIGGERED and NOTTRIGGERED. We start in the
#     # NOTTRIGGERED state.
#     triggered = False

#     index=-1
#     voiced_frames = []
#     for frame in frames:

#         index+=1
#         is_speech = vad.is_speech(frame.bytes, sample_rate)

#         sys.stdout.write('1' if is_speech else '0')
#         if not triggered:
#             ring_buffer.append((frame, is_speech))
#             num_voiced = len([f for f, speech in ring_buffer if speech])
#             # If we're NOTTRIGGERED and more than 90% of the frames in
#             # the ring buffer are voiced frames, then enter the
#             # TRIGGERED state.
#             if num_voiced > 0.9 * ring_buffer.maxlen:
#                 triggered = True
#                 sys.stdout.write('+(%s)' % (ring_buffer[0][0].timestamp,))
#                 # We want to yield all the audio we see from now until
#                 # we are NOTTRIGGERED, but we have to start with the
#                 # audio that's already in the ring buffer.

#                 id = 0 #we must start actually with num_padding_frames-1 and do index-- 
#                 for f, s in ring_buffer:
#                     voiced_frames.append(f)
#                     vuv_frames[index-id]=1
#                     id+=1

#                 ring_buffer.clear()
#         else:
#             # We're in the TRIGGERED state, so collect the audio data
#             # and add it to the ring buffer.
#             voiced_frames.append(frame)
#             vuv_frames[index] = 1
#             ring_buffer.append((frame, is_speech))
#             num_unvoiced = len([f for f, speech in ring_buffer if not speech])
#             # If more than 90% of the frames in the ring buffer are
#             # unvoiced, then enter NOTTRIGGERED and yield whatever
#             # audio we've collected.
#             if num_unvoiced > 0.9 * ring_buffer.maxlen:
#                 sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
#                 triggered = False
#                 yield b''.join([f.bytes for f in voiced_frames])
#                 ring_buffer.clear()
#                 voiced_frames = []
#     if triggered:
#         sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
#     sys.stdout.write('\n')
#     # If we have any leftover voiced audio when we run out of input,
#     # yield it.
#     if voiced_frames:
#         yield b''.join([f.bytes for f in voiced_frames])


In [5]:
# def main():
#     # if len(args) != 2:
#     #     sys.stderr.write(
#     #         'Usage: silenceremove.py <aggressiveness> <path to wav file>\n')
#     #     sys.exit(1)
#     path = '/content/amicorpus/'
#     dir_list = sorted(os.listdir(path))
#     cnt = 0
#     for d in dir_list:
#       dir_name = join(path,d)
#       if isdir(dir_name):
#         filePath = join(path, d, 'audio', d+'.Mix-Headset.wav')
#         try:
#           audio, sample_rate = read_wave(filePath)
#           vad = webrtcvad.Vad(int(1))
#           frames = frame_generator(30, audio, sample_rate)
#           frames = list(frames)
#           nof_frames = 1+(len(audio)-1)/int(sample_rate * (frame_duration_ms / 1000.0) * 2)
#           vuv_frames = np.zeros((nof_frames],)) 
#           segments = vad_collector(sample_rate, 30, 300, vad, frames,vuv_frames)

#           # Segmenting the Voice audio and save it in list as bytes
#           concataudio = [segment for segment in segments]

#           joinedaudio = b"".join(concataudio)
#           writePath = join('/content/drive/My Drive/IB_Diarization/amicorpus_non_silence', d, 'audio')
#           Path(writePath).mkdir(parents=True, exist_ok=True)
#           write_wave(join(writePath, d+'.Mix-Headset.wav'), joinedaudio, sample_rate)
#           cnt += 1
#           # if(cnt == 2):
#           #   break
#         except:
#           print("Skipping: ", filePath)
#     print("Converted: ",cnt)
# if __name__ == '__main__':
#     main()

In [6]:
# rm -rf amicorpus_non_silence/

In [7]:
# !chmod 755 amiBuild-13720-Mon-Aug-31-2020.wget.sh

In [8]:
# !ls -l '/content/drive/My Drive/amicorpus_non_silence' | wc -l

In [9]:
# !./amiBuild-13720-Mon-Aug-31-2020.wget.sh

In [10]:
# !tar -czvf filename.tar.gz amicorpus

In [11]:
# !cp filename.tar.gz /content/drive/My\ Drive/amicorpus.tar.gz

In [12]:
# !tar -xzvf /content/drive/My\ Drive/amicorpus.tar.gz

In [13]:
# !./ComputeFeatures mfcc.config /content/drive/My\ Drive/amicorpus_non_silence/ES2002b/audio/ES2002b.Mix-Headset.wav frameCepstrum+frameDeltaCepstrum sa1.mfcc 0.06 A

In [14]:
# !chmod 755 ComputeFeatures

In [15]:
# !chmod 755 mfcc.config

In [16]:
####################################
### Actual Code Starts from Here ###
####################################

In [17]:
%cd '/content/drive/My Drive/IB_Diarization/amicorpus_non_silence/ES2002b/audio/'
!ls
%cd /content

/content/drive/My Drive/IB_Diarization/amicorpus_non_silence/ES2002b/audio
ES2002b.Mix-Headset.wav
/content


In [18]:
#Define path to wav files created after VAD
path = '/content/drive/My Drive/IB_Diarization/amicorpus_non_silence/'

overlap = 0.01 #10 ms window shift
fullPath = join(path,'ES2002b/audio/ES2002b.Mix-Headset.wav')
(rate,sig) = wav.read(fullPath)
mfcc_feat = mfcc(sig, rate, numcep = 19, nfilt = 26, winlen=0.03, winstep=overlap)

In [19]:
n, d = mfcc_feat.shape

In [20]:
# overlap = 0.01 #10 ms window shift
init_cluster_time = 2500 #2.5sec
init_cluster_len = math.ceil(init_cluster_time/(overlap*1000))

In [21]:
num_of_clusters = math.ceil(n/init_cluster_len)

In [22]:
num_of_clusters

812

In [23]:
t = np.array_split(mfcc_feat, 3)

In [24]:
class GMM:
    def __init__(self, num_of_clusters):
        self.num_of_clusters = num_of_clusters
        self.log_likelihood =[]
        self.LL_diff = []
        # self.num_of_speakers = num_of_speakers

    def gaussian_prob(self, x, mean, sigma):
        d = x.shape[0]
        p = ((2*math.pi)**(-d/2))*(np.linalg.det(sigma)**(-0.5))*np.exp(-0.5*(x-mean).reshape(d,1).T.dot(np.linalg.inv(sigma)).dot((x-mean).reshape(d,1)))
        return p

    def k_means(self, X):
        n = X.shape[0]
        d = X.shape[1]
        itr = 0
        #self.centroid = np.zeros((self.num_of_clusters, d), dtype = 'float64')
        self.centroids = X[random.sample(range(n), self.num_of_clusters)]
        self.cluster_assigned = np.zeros(n, dtype = int)
        error = 0.0
        while True:
            print("Now at itr - ", itr)
            # print("Centroids - ", self.centroids)
            for i in range(n):
                f_vec = X[i]
                dist = np.sqrt(np.sum((f_vec-self.centroids)**2, 1))
                # print("Dist Shape is - ", dist.shape)
                self.cluster_assigned[i] = np.argmin(dist)
            new_error = np.sum(np.sqrt(np.sum((X - self.centroids[self.cluster_assigned])**2, 1)))
            if(itr>0):
                print("Error Difference is - ", np.abs(error-new_error))
            new_centroids = np.zeros((self.num_of_clusters, d), dtype = 'float64')
            count_of_elements = np.zeros(self.num_of_clusters, dtype = int)
            for i in range(n):
                c_ind = self.cluster_assigned[i]
                new_centroids[c_ind] += X[i]
                count_of_elements[c_ind] += 1
            new_centroids = new_centroids/count_of_elements[:,None]
            if np.abs(new_error-error)<10 or np.array_equal(self.centroids, new_centroids) or itr>=5:
                print("Breaking at itr - ", itr)
                break
            else:
                self.centroids = np.copy(new_centroids)
            itr += 1
            error = new_error

    def EM_GMM_INBUILT(self, X):
        N = X.shape[0]
        d = X.shape[1]
        from sklearn.mixture import GaussianMixture as GMM
        g = GMM(n_components=64, covariance_type = 'full', max_iter = 1)
        g.fit(X)
        print("Created")

    def EM_GMM(self, X):
        N = X.shape[0]
        d = X.shape[1]
        self.cov_mat = np.zeros((self.num_of_clusters, d, d), dtype = 'float64')
        self.gamma = np.zeros((N,self.num_of_clusters), dtype = 'float64')
        likelihood = np.zeros((N,self.num_of_clusters), dtype = 'float64')
        self.pi_prob = np.zeros(self.num_of_clusters, dtype = 'float64')
        self.Nk = np.zeros(self.num_of_clusters, dtype = 'float64')
        for k in range(self.num_of_clusters):
            indices = (np.argwhere(self.cluster_assigned==k)).ravel()
            X_k = X[indices]
            X_k_centered = X_k - self.centroids[k]
            self.Nk[k] = X_k.shape[0]
            # print("Xk ",X_k.shape)
            # print("Xkc ",X_k_centered.shape)
            # print("cov mat ",self.cov_mat[k])
            self.cov_mat[k] = (1/self.Nk[k])*(X_k_centered.T.dot(X_k_centered))
        # print(self.Nk)
        self.pi_prob = self.Nk/N
        print("EM Begins")
        itr = 1
        prev_log_likelihood = 0.0
        
        while True:
            #####################################
            ############   E Step   #############
            #####################################
            for k in range(self.num_of_clusters):
                #self.gamma[i,k] = self.gaussian_prob(X[i], self.centroids[k], self.cov_mat[k])
                self.cov_mat[k] += 1e-6*np.identity(d)
                likelihood[:,k] =  multivariate_normal.pdf(X, self.centroids[k], self.cov_mat[k]).ravel()
                # print("Done ", k)
            # log_likelihood = np.sum(np.sum((likelihood*self.pi_prob), axis = 1))

            # for i in range(N):
            #     print("Done ",i)
             
            self.gamma = likelihood*self.pi_prob
            self.gamma = self.gamma/(np.sum(self.gamma, axis = 1)[:,None])
            # print("E done")

            #####################################
            ############   M Step   #############
            #####################################
            self.Nk = np.sum(self.gamma, axis = 0)
            self.pi_prob = self.Nk/N
            for k in range(self.num_of_clusters):
                self.centroids[k] = (1/self.Nk[k])*np.sum((X*self.gamma[:,k][:,np.newaxis]), axis = 0)
                X_centered = X - self.centroids[k]
                self.cov_mat[k] = (1/self.Nk[k])*((X_centered*self.gamma[:,k][:,np.newaxis]).T.dot(X_centered))
            # print("M done")

            #####################################
            ########   Log Likelihood   #########
            #####################################
            new_log_likelihood = np.sum(np.log(np.sum((likelihood*self.pi_prob), axis = 1)))
            self.log_likelihood.append(new_log_likelihood)
            diff_LL = np.abs(new_log_likelihood-prev_log_likelihood)
            self.LL_diff.append(diff_LL)
            print("Itr = ", itr, " Current LL is - ",new_log_likelihood)
            print("Change In LL is - ",diff_LL)
            if(diff_LL<100 or itr>=10):
                print("EM Finished at iteration - ", itr)
                break
            itr += 1
            prev_log_likelihood = new_log_likelihood

In [25]:
# ug = GMM(num_of_clusters)
# ug.k_means(mfcc_feat)
# ug.EM_GMM(mfcc_feat)

In [26]:
def fitUnimodal(C):
  means = []
  covMatrices = []
  for c in C:
    means.append(np.mean(c, axis = 0))
    covMatrices.append(np.cov(c.T))
  return means, covMatrices

In [27]:
def calc_prob(x, GaussianMeans, GaussianCovMatrices):
  p = 0.0
  D = x.shape[0]
  numOfClusters = len(GaussianMeans)
  for i in range(D):
    s = x[i]
    for k in range(numOfClusters):
    #self.gamma[i,k] = self.gaussian_prob(X[i], self.centroids[k], self.cov_mat[k])
      cov_matrix = 1e-6*np.identity(d) + GaussianCovMatrices[k]
      # cov_matrix = 
      p =  p + ug.pi_prob[k]*multivariate_normal.pdf(s, ug.centroids[k], cov_matrix)
  p = p/D
  return p

In [28]:
def calcYgivenX(x, GaussianMeans, GaussianCovMatrices, i):
  p = 0.0
  numOfClusters = len(GaussianMeans)
  w = 1.0/numOfClusters
  D = x.shape[0]
  probMat = np.zeros((D, num_of_clusters), dtype = float)
  for i in range(num_of_clusters):
    probMat[:,i] = multivariate_normal(x, GaussianMeans[i], GaussianCovMatrices[i])
  p = 0.0
  self.gamma = self.gamma/(np.sum(self.gamma, axis = 1)[:,None]) 
  return p

In [29]:
########################
##### IB Algorithm #####
########################

#Init Variables
N = num_of_clusters
C = np.array_split(mfcc_feat, num_of_clusters)
GaussianMeans, GaussianCovMatrices = fitUnimodal(C)
ClusterMapping = dict(zip(range(num_of_clusters), [[i] for i in range(num_of_clusters)]))


In [30]:
probC = []
for i in range(N):
  p = 0.0
  D = C[i].shape[0]
  for j in range(D):
    s = C[i][j]
    p += multivariate_normal.pdf(s, GaussianMeans[i], GaussianCovMatrices[i])
  p = p/D 
  probC.append(p)

In [31]:
probC = (1.0/N)*np.ones(N)
probX = probC.copy()

In [39]:
beta = 10.0

In [ ]:
probYgivenC = []
probCgivenX = []
for i in range(N):
  temp1 = []
  temp2 = []
  x = C[i]
  w = 1.0/num_of_clusters
  D = x.shape[0]
  probMat = np.zeros((D, num_of_clusters), dtype = float)
  for j in range(num_of_clusters):
    probMat[:,j] = multivariate_normal.pdf(x, GaussianMeans[j], GaussianCovMatrices[j]).ravel()
  probMat = probMat/(np.sum(probMat, axis = 1)[:,None])
  temp1 = np.mean(probMat, axis = 0)
  for j in range(N):
    # p = probMat[i,j]/(np.sum(probMat[i,j], axis = 1)[:,None])
    # p = calcYgivenX(x, GaussianMeans, GaussianCovMatrices, i)
    # temp1.append(p)
    if j == i:
      temp2.append(1.0)
    else:
      temp2.append(0.0)
    # print("Done2 ",j)
  probYgivenC.append(temp1)
  probCgivenX.append(temp2)
  if i%100 == 0:
    print("Done ",i)

# # prob_cond_y_c = np.zeros((N, N), dtype = float)
# # prob_cond_c_x = np.zeros((N, N), dtype = float)
# del_F = np.zeros((N, N), dtype = float)
# for i in range(N):
#   prob_c(i) = calc_prob(C[i], ug)
#   for j in range(N):
#     prob_cond_y_c[j][i] = calc_cond_prob(j, C[i], ug)
#     if(j == i):
#       prob_cond_c_x[j][i] = 1



#Main Algo


In [ ]:
del_F = np.zeros((N, N), dtype = float)
del_F[:,:] = np.inf
probXgivenC = ((np.array(probCgivenX)*np.array(probX)).T/probC).T
for i in range(N):
  for j in range(i+1, N): 
    temp1 = distance.jensenshannon(np.array(probYgivenC)[:,i], np.array(probYgivenC)[:,j]) 
    temp2 = distance.jensenshannon(probXgivenC[i], probXgivenC[j]) 
    dij = temp1 - (1/beta)*temp2
    del_F[i][j] = (probC[i] + probC[j])*dij
    # del_F[i][j] = cal_objective_diff(C[i], C[j])
  if i%100 == 0:
    print("Done ",i)

In [32]:
!cp  /content/drive/My\ Drive/IB_Diarization/*.sav /content/
# !cp /content/*.sav  /content/drive/My\ Drive/IB_Diarization/

In [ ]:
import pickle
file_name = "probYgivenC.sav"
pickle.dump(probYgivenC, open(file_name, 'wb'))
file_name = "probCgivenX.sav"
pickle.dump(probCgivenX, open(file_name, 'wb'))
file_name = "del_F.sav"
pickle.dump(del_F, open(file_name, 'wb'))

In [34]:
import pickle

file_name = "probYgivenC.sav"
probYgivenC = pickle.load(open(file_name, 'rb'))
file_name = "probCgivenX.sav"
# pickle.dump(probCgivenY, open(file_name, 'wb'))
probCgivenX = pickle.load(open(file_name, 'rb'))
file_name = "del_F.sav"
del_F = pickle.load(open(file_name, 'rb'))

In [35]:
multivariate_normal.pdf(C[0][0], GaussianMeans[0], GaussianCovMatrices[0])

4.821832164906279e-30

In [40]:
#IB ALgo
N = num_of_clusters
# print("Yaha")
while num_of_clusters>4:
  # print("Here")
  # i, j = np.argwhere(del_F == np.min(del_F)).ravel()
  mIdx = np.argmin(del_F)
  i = mIdx//N
  j = mIdx%N
  probCr = probC[i] + probC[j]
  del_F[:,j] = np.inf
  del_F[j,:] = np.inf
  # probC.pop(j)
  ClusterMapping[i] += ClusterMapping[j]
  ClusterMapping[j] = []
  probYgivenC[i] = (probYgivenC[i]*probC[i] + probYgivenC[j]*probC[j])/probCr
  probC[i] = probCr
  probCgivenX[i] = [0 for idx in probCgivenX[i]]
  for idx in ClusterMapping[i]:
    probCgivenX[i][idx] = 1
  probXgivenC = ((np.array(probCgivenX)*np.array(probX)).T/probC).T
  for idx in range(0, i):
    if del_F[idx,i] == np.inf:
      continue
    temp1 = distance.jensenshannon(np.array(probYgivenC)[:,idx], np.array(probYgivenC)[:,i]) 
    temp2 = distance.jensenshannon(probXgivenC[idx], probXgivenC[i]) 
    dij = temp1 - (1/beta)*temp2
    del_F[idx][i] = (probC[idx] + probC[i])*dij
  for idx in range(i+1, N):
    if del_F[i, idx] == np.inf:
      continue 
    temp1 = distance.jensenshannon(np.array(probYgivenC)[:,i], np.array(probYgivenC)[:,idx]) 
    temp2 = distance.jensenshannon(probXgivenC[i], probXgivenC[idx]) 
    dij = temp1 - (1/beta)*temp2
    del_F[i][idx] = (probC[i] + probC[idx])*dij
  num_of_clusters = num_of_clusters-1
  if num_of_clusters%100 == 0:
    print("Clusters Rem: ", num_of_clusters)


Clusters Rem:  800
Clusters Rem:  700
Clusters Rem:  600
Clusters Rem:  500
Clusters Rem:  400
Clusters Rem:  300
Clusters Rem:  200
Clusters Rem:  100


In [8]:
file_name = "ClusterMapping.sav"
import pickle
pickle.dump(ClusterMapping, open(file_name, 'wb'))

NameError: ignored

In [9]:
!cp /content/*.sav  /content/drive/My\ Drive/IB_Diarization/


In [5]:
# Start Prob [1,0,0,0] or [0.25]*4
#transistion prob [0.25]
from hmmlearn import hmm

clstrs = []
for i in range(N):
  if len(ClusterMapping[i]) !=0:
    clstrs.append(ClusterMapping[i]);

clstr_seq = np.ones((N,))*-1
# --------------------------------------------------------------------------
start_prob=np.ones((4,))/4.0
# --------------------------------------------------------------------------
id = 0;
max_clstr = 0
for clstr in clstrs:
  id+=1
  if len(clstr)>max_clstr:
    max_clstr = len(clstr)
hmm_wts = np.zeros((4,max_clstr)).astype('float')
hmm_means = np.zeros((4,max_clstr,mfcc_feat.shape[-1]))
hmm_covar = np.zeros((4,max_clstr,mfcc_feat.shape[-1],mfcc_feat.shape[-1]))

id=-1
for clstr in clstrs:
  id+=1
  clstr_wt = 1.0/len(clstr)
  inclstr_id=0
  for seg in clstr:
    clstr_seq[inclstr_id]=id
    hmm_wts[id][inclstr_id]=clstr_wt
    hmm_means[id][inclstr_id]=GaussianMeans[seg]
    hmm_covar[id][inclstr_id]=GaussianCovMatrices[seg]
    inclstr_id+=1

trans_prob = np.zeros((4,4))
# --------------------------------------------------------------------------
clstr_seq = clstr_seq.astype(np.int)
for i in range(1,N):
  trans_prob[clstr_seq[i-1]][clstr_seq[i]]+=1
trans_prob = np.divide(trans_prob.astype('float'),np.sum(trans_prob,axis=1))
# --------------------------------------------------------------------------
U_GMM_HMM =  hmm.GMMHMM(n_components =4,
                        n_mix =max_clstr ,
                        covariance_type = "full",
                        startprob_prior = start_prob,
                        transmat_prior  = trans_prob,
                        weights_prior  =hmm_wts ,
                        # means_weight  = hmm_means,
                        # covars_weight  = hmm_covar,
                        )
print(" Shape :  start_prob ",start_prob.shape)
print(" Shape :  trans_prob ",trans_prob.shape)
print(" Shape :  hmm_wts ",hmm_wts.shape)
print(" Shape :  hmm_means ",hmm_means.shape)
print(" Shape :  hmm_covar ",hmm_covar.shape)
print(" Shape :  mfcc_feat ",mfcc_feat.shape)
aligned_lbls = U_GMM_HMM.fit(mfcc_feat[:8000])



NameError: ignored

In [ ]:
print(U_GMM_HMM.means_.shape) 
print(U_GMM_HMM.covars_.shape) 

In [ ]:
p

In [ ]:
p.insert(1, 13)

In [ ]:
d = dict(zip(range(10),[[i] for i in range(10)]))

In [ ]:
d[0].append(2)

In [ ]:
d

In [ ]:
from scipy.stats import norm

In [ ]:
m,c = norm.fit(mfcc_feat[0:100],d=19)

TypeError: ignored

In [ ]:
data = norm.rvs(loc=0,scale=2,size=10, )

In [ ]:
c.shape

()

In [ ]:
mfcc_feat[0:100].shape

(100, 19)

In [ ]:
m = np.mean(mfcc_feat[0:100], axis = 0)

In [ ]:
c = np.cov(mfcc_feat[0:100].T)

In [ ]:
m.shape

(19,)

In [ ]:
a = 1
a +=2
a

3

In [ ]:
a = [[-10,-3,4], [4,-104,-500]]

In [ ]:
i, j = np.argwhere(a == np.min(a)).ravel()

In [ ]:
a = [1,2,3] + [3,4,5]

In [ ]:
a[0] = [9 for i in a[0]]

In [ ]:
a

[[9, 9, 9], [4, -104, -500]]

In [ ]:
import hmmlearn

ModuleNotFoundError: ignored

In [48]:
!cp  /content/drive/My\ Drive/IB_Diarization/amicorpus/ami_public_manual_1.6.2.zip /content/

In [ ]:
import xml.etree.ElementTree as ET
import numpy as np
ns = {'nite':"http://nite.sourceforge.net/", 'id':"EN2001a.A.segs"}
alls = ['A','B','C','D']
Vec = np.zeros((3000000,))
def main():
	d=0
	for i in alls:
		parse(i,d+1)
		d+=1
	np.save('Vec.npy',Vec)

def parse(S,i):	
	tree = ET.parse('./Data/EN2002b.'+S+'.segments.xml')
	root = tree.getroot()
	for seg in root.findall('segment'):
		st = seg.get('transcriber_start')
		end = seg.get('transcriber_end')
		st = int(float(st)*100)
		end = int(float(end)*100)
		for k in range(st,end):
			Vec[k]=i
if __name__ == '__main__':
	main()
	print(Vec[5000:10000],Vec.shape)